# Lab 6: Spec-driven development of a visual conversation analytics agent

The goal of this lab is to augment the existing environment with an additional agent which helps us to analyze previous conversation to improve the quality of our support center. A key prerequisite is the set up of the Q Developer CLI for building the AI Agent using the AWS Strands SDK. The setup further includes package managers, MCP servers, and development rules for spec-driven development. We use the spec-driven technique to generate the additional AI agent.

## Prerequisites
- AWS Builder ID. Free sign-up: https://builder.aws.com/start
- Previous labs executed until Lab 4
- Internet connection for downloading packages


Execute all the steps in the JupyterLab Termnial. Therefore, open a new Terminal session by navigating to the launcher page and start a fresh Terminal session

![alt text](./img/open_terminal.png)
---

## Step 1: Install UV Package Manager

UV is a fast Python package manager that we'll use to manage dependencies and run tools. This command downloads and installs UV, then reloads the shell to make it available in the current session.

In [ ]:
curl -LsSf https://astral.sh/uv/install.sh | sh && exec $SHELL

Ignore warnings/errors that the uv or uvx is already configured for this shell.

Now we need to start Q Developer CLI the first time and login into the AWS builders account

In [ ]:
q login

You see see a scree like this. Choose Free Builder ID.

![alt text](./img/q_login.png)

Open the link in a fresh browser tab and follow the instruction to login until you see that screen

![alt text](./img/q_login_success.png)

Congratulations, the Q CLI is now ready to use!

## Step 2: Configure Strands MCP Server

This step sets up the MCP (Model Context Protocol) configuration environment for the Q CLI and specifically the Strands Agents MCP server. The configuration:

- Creates the Q CLI configuration directory
- Configures the Strands MCP server to run via `uvx`
- Sets logging level to INFO for debugging
- Auto-approves safe documentation tools (`search_docs`, `fetch_doc`)

The MCP server provides access to Strands SDK documentation and examples directly within Q CLI.

In [ ]:
# Create the MCP configuration directory if it doesn't exist
mkdir -p ~/.aws/amazonq

# Add the Strands MCP server configuration to Q CLI
cat > ~/.aws/amazonq/mcp.json << 'EOF'
{
  "mcpServers": {
    "strands-agents": {
      "command": "uvx",
      "args": ["strands-agents-mcp-server"],
      "env": {
        "FASTMCP_LOG_LEVEL": "INFO"
      },
      "disabled": false,
      "autoApprove": [
        "search_docs",
        "fetch_doc"
      ]
    }
  }
}
EOF

## Step 3: Add AWS Documentation MCP Server

Add the AWS Documentation MCP server to Q CLI. It provides access to official AWS services documentation. This is essential for understanding AWS services and their APIs while developing agents.

In [ ]:
q mcp add --name aws-documentation --command uvx --args "awslabs.aws-documentation-mcp-server@latest" --env "FASTMCP_LOG_LEVEL
=ERROR,AWS_DOCUMENTATION_PARTITION=aws"

## Step 4: Add Amazon Bedrock AgentCore MCP Server

This step adds the Amazon Bedrock AgentCore MCP server, which provides tools for:

- Managing Bedrock agents and knowledge bases
- Querying knowledge bases with natural language
- Accessing AgentCore deployment capabilities

The `FASTMCP_LOG_LEVEL=ERROR` setting reduces log verbosity for cleaner output.

In [ ]:
q  mcp add --name amazon-bedrock-agentcore-mcp-server --command uvx --args "awslabs.amazon-bedrock-agentcore-mcp-server@latest" --env FASTMCP_LOG_LEVEL=ERROR

## Step 5: Download and Setup Development Rules

This step downloads the development rules file that defines:

- Project structure standards for Strands agents
- Development workflow guidelines
- Code generation constraints
- Testing requirements

The rules file is placed in the Q CLI rules directory to enable spec-driven development practices.

In [ ]:
wget https://raw.githubusercontent.com/Klosephi/spec-driven-strands/refs/heads/main/rule.md
mkdir -p .amazonq/rules
mv rule.md .amazonq/rules/

## Step 6: Validate the Setup

Let's validate that we successfully installed the tools and configuration. Start the Q CLI and see if our MCP servers a getting started. When we do this the first time it might take some time for Q to load and install all the tools you might find a warning.

In [ ]:
q chat

We can at anytime check the status of our tools by typing /tools insinde the chat

In [ ]:
/tools

![alt text](./img/q_started.png)

---

## Understanding Q CLI Rules and Spec-Driven Development

### What are Q CLI Rules?

**Q CLI Rules** are markdown files that define project-specific guidelines, coding standards, and behavioral rules for Amazon Q Developer CLI. These rules are automatically integrated into agent contexts, ensuring consistent behavior across your development workflow.

**Key Features:**
- **Workspace-specific**: Rules only apply to the current project directory
- **Automatic integration**: Default agent includes all rules from `.amazonq/rules/**/*.md`
- **Context loading**: Rules become part of the agent's context, influencing all interactions
- **Markdown format**: Standard markdown files with headers, code blocks, and examples

**Reference:** [AWS Documentation - Project Rules](https://docs.aws.amazon.com/amazonq/latest/qdeveloper-ug/command-line-project-rules.html)

### How Rules Work

1. **Storage**: Rules are stored in `.amazonq/rules/` directory as `.md` files
2. **Loading**: Q CLI automatically loads all rules when starting an agent session
3. **Context**: Rules become part of the agent's system context
4. **Influence**: Every code generation and suggestion follows the defined rules

### Rule Types

**Coding Standards:**
```markdown
# Python Coding Standards
- Use Black for code formatting
- Maximum line length: 88 characters
- Use type hints for all functions
```

**Architecture Guidelines:**
```markdown
# Architecture Guidelines
- Use microservices pattern for new features
- Implement circuit breaker for external APIs
- All services must expose health endpoints
```

**Security Policies:**
```markdown
# Security Guidelines
- All API endpoints require authentication
- Never log sensitive information
- Use parameterized queries
```
### What is Spec-Driven Development?

**Spec-Driven Development** is a methodology where development is guided by explicit specifications and rules that define:

- **Project Structure**: Standardized directory layouts and file organization
- **Development Workflow**: Step-by-step processes for building features
- **Code Generation**: Constraints and patterns for consistent code output
- **Testing Requirements**: Mandatory testing practices and coverage

### Benefits of Spec-Driven Development

1. **Consistency**: All team members follow the same patterns
2. **Quality**: Built-in best practices and standards
3. **Efficiency**: Reduced decision-making overhead
4. **Maintainability**: Predictable code structure and patterns
5. **Onboarding**: New developers can quickly understand project conventions

### How It Works with Strands SDK

The rules file we downloaded defines specific patterns for **AWS Strands Agent development**:

**Project Structure:**
```
project/
├── infra/           # Jupyter notebooks for AWS infrastructure
├── src/
│   ├── agents/      # Strands agent implementations
│   └── tools/       # Custom Python tools
└── tests/           # Unit and integration tests
```

**Development Workflow:**
1. **Research Phase**: Check PyPI for latest Strands SDK, validate existing tools
2. **Incremental Development**: Small, testable chunks with simple implementations
3. **Code Generation**: Precise scope, no assumptions about unstated requirements

---

## Working with Q CLI - Getting Started

After completing the setup, you're ready to start using Q CLI for spec-driven development. Here's how to work with Q CLI effectively:

### How Q CLI Works

**Q CLI** is an interactive AI assistant that:
- Automatically loads your project rules from `.amazonq/rules/`
- Uses MCP servers to access documentation and tools
- Provides context-aware code generation and guidance
- Can execute commands and create files in your project

### Starting Q CLI

1. **Open terminal** in your project directory
2. **Run:** `q chat`
3. **Wait** for Q CLI to load rules and MCP servers
4. **Start chatting** with natural language prompts

### Tool Trust and Permissions

When Q CLI wants to use tools (create files, run commands), it will:
- **Ask for permission** before executing actions
- **Show you** exactly what it plans to do
- **Wait for approval** before proceeding

**Common responses:**
- Type `y` or `yes` to approve
- Type `n` or `no` to decline
- Type `always` to auto-approve similar actions
- Type `never` to block similar actions

### Best Practices
- **Review** what Q CLI plans to do before approving
- **Start small** with simple requests to build trust
- **Be specific** in your prompts for better results
- **Use incremental** development approach

*Q CLI Commands:**
```bash
# Start interactive chat
q chat

# List available MCP servers
q mcp list

# Get help
q --help

# Exit Q CLI
/quit
```

---
## Step 7: Vibe-code the conversation analytics Agent

We want to build an agent which helps us to analyze previous conversation to improve the quality of our support center. Therefore we ask Q CLI to understand the current context where and how to deploy the agent to first. Then we provide the requirements.

Please Note:
- It will ask for allowance if it's going to use the tool the first time. You can trust it.
- If the agents waits and does not clearly ask for input allowance, tell the agent to go on.
- Experiment! This is not a fully optimized example in a way that you copy and paste it, success means that the agent follows the rules we defined upfront to execute the task as we defined it and you iterate it.
- If the session is summarized remind the CLI to continue with the plan

Start (or continue) the Q CLI chat:

In [ ]:
q chat

Now let's deploy and see spec-driven development using the Q CLI in action! Copy & paste the prompt below into your active Q CLI chat to start the process:

**Remember:** The Q CLI asks for review and confirmation to proceed hence it waits for your input. The overall process completes in about 15min. Watch how it executes the plan, runs tests, fixes the code and redeploys as needed.

In [ ]:
I want to create a very simple Visual Analytics Agent for the Customer support application. This agent is an extension of the existing strands application. 
To understand the application we want to extend understand these notebooks bedrock-agentcore-workshop/lab-04-agentcore-runtime.ipynb and bedrock-agentcore-workshop/lab-03-agentcore-gateway.ipynb.


The Agent must generate real-time visual insights from conversation data. 
** Important:** 
    - Reuse the existing Agentcore setup, Reuse the execution role from the previous deployment.

**Requirements:**
- Deploy as independent Agent (no modifications to existing customer support agent)
- Mock the conversation history instead of using real data
- I want to understand how many conversations the agent had with how many clients. Instead of text response I want to have a visually appealing chart
- Enable usage and application logs for the agent

### Result
Once the process completes, the Q CLI provides you a summary. Explore the generated artifacts. You will notice that it followed the rules and specifications, integrated into the existing AgentCore Runtime and Identity provider as your existing Customer Support Agent. You can browse in the AWS Console to the Bedrock AgentCore service to verify the deployment. To interact with the newly created Visual Analytics Agent, you can either enhance the created notebook or simply ask Q CLI to build you a notebook so that you can test the agent with your own queries.

![alt text](./img/conversation_analytics_agent.png)

Exemplary results of the Visual Analytics Agent using mock conversation data:
![alt text](./img/exemplary_results.png)

---

## References

### AWS Documentation
- [Amazon Q Developer Project Rules](https://docs.aws.amazon.com/amazonq/latest/qdeveloper-ug/command-line-project-rules.html)
- [Amazon Q Developer Features](https://docs.aws.amazon.com/amazonq/latest/qdeveloper-ug/features.html)
- [AWS Prescriptive Guidance - Developer Workflows](https://docs.aws.amazon.com/prescriptive-guidance/latest/best-practices-code-generation/developer-workflows.html)

### Strands SDK Documentation
- [Strands Multi-Agent Patterns](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/multi-agent-patterns/)
- [Agent2Agent (A2A) Protocol](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/agent-to-agent/)
- [CLI Reference Implementation](https://strandsagents.com/latest/documentation/docs/examples/python/cli-reference-agent/)

### Tools and Protocols
- [Model Context Protocol (MCP)](https://modelcontextprotocol.io/)
- [UV Package Manager](https://docs.astral.sh/uv/)
- [Agent2Agent Protocol](https://a2aproject.github.io/A2A/latest/)